Tony Chouteau 

Info2

2019 - 2020

---

Jupyter - Google Collab



#**Introduction**

####**Imports**

In [0]:
import random as rand
import sys
import sympy as sy
import time
import copy
#import math

####**Définition**

J'utilise la bibliothèque $sys$ pour augmenter le nombre maximum de récursion (utile pour la génération de très grands nombres premiers).

J'utilise la bibliothèque $sympy$ pour l'ensemble de ses fonctions permettant de trouver, vérifier des nombres premiers, et donc tester mes algorithmes.

J'utilise la bibliothèque $time$ pour mesurer la complexité temporelle réelle de mess algorithmes. J'en profite pour définir une fonction $millis()$ qui renvoie le nombre de millisecondes depuis "epoch", soit le *1er janvier 1970, 00:00:00 (UTC)*

In [0]:
print("La version de python utilisé est :", sys.version)
print("")

limit = 10**7
print("Nombre de récursions maximum définie à :",limit)
sys.setrecursionlimit(limit)
print("")

millis = lambda: int(round(time.time() * 10**3))
print("Milliseconde actuelle :",millis())

micro = lambda: int(round(time.time() * 10**6))
print("MicroSeconde actuelle :",micro())

def displayPeriod(t):
  if (t<10**3):
    return (str(t)+"µs")
  if (t<10**6):
    return (str(t/10**3)+"ms")
  else:
    return (str(t/10**6)+"s")

La version de python utilisé est : 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]

Nombre de récursions maximum définie à : 10000000

Milliseconde actuelle : 1587401867651
MicroSeconde actuelle : 1587401867650864


#**Partie 1 - Arithmétique dans Z et Z/nZ avec Python**

##**Question 1**

###**Euclide Simple**

####**Définition**

In [0]:
def euclideSimple(a, b):
  assert (a > 0 and b > 0), "a et b doivent être positifs"
  
  if a < b:
    a, b = b, a

  a0, b0 = a, b
  
  while b:
    a, b = b, a % b
  
  return a, a0//a, b0//a

####**Test**

In [0]:
a = 60
b = 45

r, a0, b0 = euclideSimple(a, b)

print("Le PGCD de", a, "et de", b, "est :",r)
print("")
print("Ainsi : ")
print(a0, "*", r, "=", a)
print(b0, "*", r, "=", b)

Le PGCD de 60 et de 45 est : 15

Ainsi : 
4 * 15 = 60
3 * 15 = 45


###**Euclide Étendu**

####**Définition**

In [0]:
def euclideEtendu(a, b):
  assert type(a)==type(b), "Les deux paramêtres doivent avoir le même type"
  
  r, u, v, r0, u0, v0 = a, 1, 0, b, 0, 1
  
  while r0:
    q = r//r0
    r, u, v, r0, u0, v0 = r0, u0, v0, r-q*r0, u-q*u0, v-q*v0

  return r, u, v

####**Test**

In [0]:
a = 45
b = 60

r, u, v = euclideEtendu(a, b)

print("Le PGCD de", a, "et de", b, "est :",r)
print("")
print("De plus : r=au+bv avec u =",u," et v =",v)
print("")
print(r, "=", a, "*", u, "+", b, "*", v, "=", a*u, "+", b*v)

Le PGCD de 45 et de 60 est : 15

De plus : r=au+bv avec u = -1  et v = 1

15 = 45 * -1 + 60 * 1 = -45 + 60


##**Question 2**

###**Inverse Modulaire**

####**Définition**

In [0]:
def inverseMod(x, n):
  r, u, _ = euclideEtendu(x, n)
  
  assert r==1, "x et n doivent être premiers entre eux" 

  return u%n

####**Test**

In [0]:
a = 23
n = 29

inv = inverseMod(a, n)

print("L'inverse de",a,"modulo",n,"est :",inv)

L'inverse de 23 modulo 29 est : 24


##**Question 3**

###**Exponentiation Rapide**

####**Définition**

In [0]:
def expoRapide(x, k, n=None):
  if (k == -1 and n != None): #Inverse modulaire
    return inverseMod(x,n)
  if (k == 1):
    if (n!=None):
      return x%n
    else:
      return x
  if (not k%2): # Paire
    #print(x, k, k/2, k//2, type(k), "paire")
    if (n!=None):
      return expoRapide(x*x%n, k//2, n)%n
    else:
      return expoRapide(x*x, k//2, n)
  if (k%2): # Impaire
    #print(x, k, k/2, k//2, type(k), "impaire")
    if (n!=None):
      return x*expoRapide(x*x%n, (k-1)//2, n)%n
    else:
      return x*expoRapide(x*x, (k-1)//2, n)

####**Test**

In [0]:
x = 4
k = 87

r = expoRapide(x, k)

print(x,"à la puissance",k,"donne :",r)

4 à la puissance 87 donne : 23945242826029513411849172299223580994042798784118784


##**Question 4**

###**IntegerModRing**

Utilisant Jupyter je n'ai pas accès à l'objet $IntegerModRing$ de SAGE, je ne peux donc pas proposer d'études temporelle.


Je l'ai tout de même re-créé en utilisant mes méthodes.

####**Définition**

In [0]:
class IntegerModRing: #Z/nZ
  
  def __init__(self, n): # Constructeur : IntegerModRing(n)
    self.n = n
    self.__call__.n = self.n

  class __call__: # Rendre l'objet "Callable" : A(x)
      def __init__(self, x):
        self.value = x % self.n
      def __add__(self, a): # Redéfinition de la puissance
        return (self.value+a) % self.n
      def __mul__(self, multi): # Redéfinition de la multiplication
        return (self.value * multi) % self.n
      def __pow__(self, power): # Redéfinition de la puissance
        return expoRapide(self.value, power, self.n)

####**Test**

In [0]:
x = 11
k = 2

n = 5

print("A = IntegerModRing(",n,")")
A = IntegerModRing(n)
print("")

r = A(x)+k
print("Addition A(x)+k :\n",x,"%",n,"=",r)
print("")

r = A(x)*k
print("Multiplication A(x)**k :\n",x,"*",k,"%",n,"=",r)
print("")

r = A(x)**k
print("Puissance A(x)*k :\n",x,"^",k,"%",n,"=",r)
print("")

r = A(x)**(-1)
print("Inverse A(x)**(-1) :\n",x,"^ -1 %",n,"=",r)

A = IntegerModRing( 5 )

Addition A(x)+k :
 11 % 5 = 3

Multiplication A(x)**k :
 11 * 2 % 5 = 2

Puissance A(x)*k :
 11 ^ 2 % 5 = 1

Inverse A(x)**(-1) :
 11 ^ -1 % 5 = 1


#**Partie 2 - Codage et Décodage**

##**Question 1**

###**Alphabet et Dictionnaires**

####**Définition**

In [0]:
abc = "$\n abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890,.;:!?'()&%-+*/=@"

cDico = {abc[key]:key for key in range(len(abc))}
dDico = {key:abc[key] for key in range(len(abc))}

####**Test**

In [0]:
print("Début \"",abc[0], "-", abc[1],"\" Fin")
print("Tailles :", len(abc), len(cDico), len(dDico))
print("")

print("Alphabet :",abc)
print("")
print("Dico de codage :",cDico)
print("Dico de décodage :",dDico)

Début " $ - 
 " Fin
Tailles : 82 82 82

Alphabet : $
 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890,.;:!?'()&%-+*/=@

Dico de codage : {'$': 0, '\n': 1, ' ': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, 'A': 29, 'B': 30, 'C': 31, 'D': 32, 'E': 33, 'F': 34, 'G': 35, 'H': 36, 'I': 37, 'J': 38, 'K': 39, 'L': 40, 'M': 41, 'N': 42, 'O': 43, 'P': 44, 'Q': 45, 'R': 46, 'S': 47, 'T': 48, 'U': 49, 'V': 50, 'W': 51, 'X': 52, 'Y': 53, 'Z': 54, '1': 55, '2': 56, '3': 57, '4': 58, '5': 59, '6': 60, '7': 61, '8': 62, '9': 63, '0': 64, ',': 65, '.': 66, ';': 67, ':': 68, '!': 69, '?': 70, "'": 71, '(': 72, ')': 73, '&': 74, '%': 75, '-': 76, '+': 77, '*': 78, '/': 79, '=': 80, '@': 81}
Dico de décodage : {0: '$', 1: '\n', 2: ' ', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i',

##**Question 2**

###**Encodage d'un bloc**

####**Définition**

Remarque : Dans le cas d'un caractère inconnu, il est remplacé par le dernier caractère de l'alphabet, rajouté pour l'occasion : "@".

In [0]:
def encodageBloc(bloc):
  
  result = []
  for c in bloc:
    try:
      result.append(cDico[c])
    except KeyError:
      result.append(len(abc)-1)

  number = 0
  for i in range(len(result)):
    number += result[i]*len(abc)**i

  return number

####**Test**

In [0]:
msg = "Hello World"
print("Le message est : \""+msg+"\"")

print("")
cMsg = encodageBloc(msg)
print("L'encodage du message donne :", cMsg)

Le message est : "Hello World"

L'encodage du message donne : 84856814622316789146


##**Question 3**

Pour une taille de bloc $P$, et un alphabet de taille $N$, le code numérique maximal d'un bloc est de $N^{P-1}$

###**Décodage d'un bloc**

####**Définition**

Remarque : Une taille de bloc est nécessaire en paramètre pour connaitre la puissance maximale de l'encodage du dit-bloc.

In [0]:
def decodageBloc(bloc, maxChar=20):
  assert maxChar<len(abc), "Taille de bloc trop grande"
  assert bloc<(len(abc)**(maxChar)), "Bloc non valide, valeur trop grande"

  result = []

  reste = bloc
  for i in range(maxChar-1, -1, -1):
    if(reste >= len(abc)**i):
      result.append(reste//(len(abc)**i))
      reste = reste%(len(abc)**i)

  msg = ""
  for n in result:
    try:
      msg = dDico[n] + msg
    except KeyError:
      msg = "@" + msg

  return msg

####**Test**

Ajout d'un caractère avant le décodage : ajout d'un $len(dDico)$ (Afin d'être toujours en dehors de l'alphabet).

*Il y a en effet deux "@" à la fin de la chaine obtenue après décodage*

In [0]:
dMsg = decodageBloc(cMsg)
print("Le decodage du message donne : \""+dMsg+"\"")
print("                Vérification : \""+msg+"\"")

Le decodage du message donne : "Hello World"
                Vérification : "Hello World"


##**Question 4**

###**Encodage d'un message entier**

####**Définition**

In [0]:
def encodage(msg, tailleBloc=20):
  
  encodedMsg = []

  for i in range(0, len(msg), tailleBloc):
    encodedMsg.append(encodageBloc(msg[i:i+tailleBloc]))

  return encodedMsg

####**Test**

In [0]:
msg = "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
print("Le message est : \""+msg+"\"")

print("")
cMsg = encodage(msg)
print("L'encodage du message complet donne :", cMsg)

Le message est : "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "

L'encodage du message complet donne : [5175605931744531058174221399946258109, 53610257152947054120140631808764154323, 103754296707586791015869521983086626148, 5228350774909902220910001040676773721, 11942020454605490003925893614395621093, 3706727044528441802558488123046976

###**Décodage d'un message codé entier (soit une liste de blocs encodés)**

####**Définition**

In [0]:
def decodage(blocs, tailleBloc=20):
  
  decodedMsg = ""

  for bloc in blocs:
    decodedMsg += decodageBloc(bloc, tailleBloc)

  return decodedMsg

####**Test**

Remarque : Le caractère special "à" à été remplacé par un "@" comme prévu


In [0]:
dMsg = decodage(cMsg)
print("Le decodage du message complet donne : \""+dMsg+"\"")
print("                        Vérification : \""+msg+"\"")

Le decodage du message complet donne : "@ Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
                        Vérification : "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim ele

#**Partie 3 - RSA**

##**Question 1**

###**Production des valeurs p, q et a**

####**Définition**

In [0]:
def bobPrive(min=10**19, max=10**20):

  p, q = 0, 0
  while q==p:
    p, q = sy.randprime(min, max), sy.randprime(min, max)

  assert (min>0 and max > min), "Le paramêtre min doit être supérieur à 0 et max doit être supérieur à min"

  a, r = p, 0
  while r!=1:
    a = rand.randint(min, (p-1)*(q-1)-1)
    r, _, _ = euclideSimple(a, (p-1)*(q-1))
  
  return p, q, a

####**Test**

#####**Partie Privé**

Si $power>14$, la fonction factor va devenir très chronophage.

In [0]:
power = 19

q, p, a = bobPrive(10**power, 10**(power+1))
print("Bob produit 2 nombres premiers :")
print("p =",p,"et q =",q)
print("")

print("Bob produit un nombre aléatoire premier avec phi(n)=(p-1)(q-1) :")
print("a =",a)

Bob produit 2 nombres premiers :
p = 94528508018390573417 et q = 72362655780662709601

Bob produit un nombre aléatoire premier avec phi(n)=(p-1)(q-1) :
a = 356663546924846595029649766226155769309


#####**Partie Public**

In [0]:
n = p*q
print("Bob publie n=pq :")
print("n =",n)
print("")

b = inverseMod(a, (p-1)*(q-1))
print("Bob publie l'inverse de a modulo phi(n) :")
print("b =",b)

Bob publie n=pq :
n = 6840333887194411923603182317831341276617

Bob publie l'inverse de a modulo phi(n) :
b = 3600195712633119537772004031213740617589


#####**Factor()**

La fonction utilisé ici est la fonction $primefactors(n)$ de la bibliothèque Sympy, le calcul est très chronophage pour des valeurs supérieures à $10^{15}$ (cf : Partie Privé/Test)

On remarque qu'il y a l'air d'y avoir que 2 facteurs premiers.

In [0]:
power = 14
q, p, a = bobPrive(10**power, 10**(power+1))
n = p*q
print(p,q)

if (power <= 15):
  l = sy.primefactors(n)
  print(l)
else:
  print("La valeur power est trop haute, le calcul durerait trop longtemps")

934263701871971 808037973730219
[808037973730219, 934263701871971]


##**Question 2**

###**Obtention la taille du bloc**

####**Définition**

In [0]:
def tailleBlocMax(n):

  tailleBloc = 0
  while len(abc)**(tailleBloc+1)<n:
    tailleBloc+=1

  return tailleBloc

####**Test**

In [0]:
print("Pour n =",n,", la taille du bloc est de maximum :",tailleBlocMax(n))

Pour n = 703151625286020377226436486973687412359 , la taille du bloc est de maximum : 20


###**Chiffrement RSA d'un simple bloc**

####**Définition**

In [0]:
def chiffrementRSA(x, n, b):
  assert x<n, "x doit être inférieur à n"

  return expoRapide(x, b, n)

####**Test**

In [0]:
msg = "HelloWorld"
cMsg = encodageBloc(msg)
rsa = chiffrementRSA(cMsg, n, b)
print("Le message \""+msg+"\" à été chiffré :",rsa)

Le message "HelloWorld" à été chiffré : 310517144444725345154752616543913229466


###**Chiffrement RSA d'un message entier**

####**Définition**

In [0]:
def messageToRSA(msg, n, b):

  taille = tailleBlocMax(n)
  l = encodage(msg, tailleBloc=taille)

  result = []
  for elt in l:
    result.append(chiffrementRSA(elt, n, b))
  
  return result

####**Test**

In [0]:
msg = "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
msgRSA = messageToRSA(msg, n, b)
print("Le message \""+msg+"\"")
print("à été chiffré :",msgRSA)

Le message "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
à été chiffré : [577011532823449721367708433375207141477, 599728297935138904149896076215791075389, 311672465081818313699432163553942603008, 541919014569821444887520151700839064327, 491373921518885616267204059484251097681, 85652247805818157110412485825694762013, 18848730977291749

##**Question 3**

###**Déchiffrement RSA d'un message**

####**Définition**

In [0]:
def rsaToMessage(rsa, n, a):

  result = []
  for elt in rsa:
    result.append(chiffrementRSA(elt, n, a))
  
  taille = tailleBlocMax(n)
  l = decodage(result, tailleBloc=taille)

  return l

####**Test**

Le message est donc bien déchiffré, exepté, encore une fois, pour le caractère spécial "à" qui a été remplacé par un "@".

In [0]:
print("Le message à déchiffrer est :",msgRSA)
print("")

msgAfter = rsaToMessage(msgRSA, n, a)
print("Message déchiffré :",msgAfter)
print("     Vérification :",msg)

Le message à déchiffrer est : [577011532823449721367708433375207141477, 599728297935138904149896076215791075389, 311672465081818313699432163553942603008, 541919014569821444887520151700839064327, 491373921518885616267204059484251097681, 85652247805818157110412485825694762013, 188487309772917494444462622358692306516, 89673345522126138502683413076976311930, 117274855642106810681179165294973817417, 385930657893566508201475228921213625411, 493859657645237713714915367596617695393, 253882879923847681017776748654756920639, 433912913315391763025210363413238742559, 74114861726670017395960187045312051301, 484939815222144249222950218806950495249, 542391856705259701714887849538197348682, 596132340838767748854736592982274781299, 54458192795383791718776278581660031149, 615906456451145700752975483091455280658, 372081530781709457261869499424448953085, 615643496742584684835574989659128634444, 97375655448521933802172471592077183830, 388830265014728915269125147836466917481, 5971619591251513781668378606240

##**Question 4**

###**Déchiffrement frauduleux**

1) $x$ doit être premier avec $n$ :

Le principe du chiffrement RSA est fondé sur la complexité de la décomposition d'un grand nombre en facteurs premiers, en d'autres mots, de retrouver $p$ et $q$ à partir de $n$. 

Si $x$ n'est pas premier avec $n$, alors $x$ vaut soit $q$, soit p ($x$ étant inférieur à $n$), ainsi si $x$ n'est pas premier, il est possible de retrouver les facteurs premiers de n qui sont donc $x$ et $\frac{n}{x}$.

2) Oscar peut-t'il trouver la clé prive

#####



####**Définition**

In [0]:
def probXPrimeWithN(n):
  
  xMax = tailleBlocMax(n)
  return "2/"+str(len(abc)**xMax)

####**Test**

In [0]:
print("La probabilité que le message encodé ne soit pas premier avec n est de :",probXPrimeWithN(n))

La probabilité que le message encodé ne soit pas premier avec n est de : 2/188919613181312032574569023867244773376


#**Partie 4 - Génération de nombres premiers**

##**Question 1**

###**Resolution de l'équation (n-1) = 2^s*d**

####**Définition**

In [0]:
def resolveMillerFormula(n):
  assert n%2, "n doit être impair"

  s = 0
  
  nMinus1Bin = bin(n-1)[2:]
  while (s<len(nMinus1Bin) and nMinus1Bin[len(nMinus1Bin)-1-s]=='0'):
    s+=1

  d = int(nMinus1Bin[:len(nMinus1Bin)-s], 2)
  
  return s, d

####**Test**

In [0]:
n = 565

s, d = resolveMillerFormula(n)
print(n-1, "= 2 ^", s, "*", d)

564 = 2 ^ 2 * 141


###**Test de Miller en base définie**

####**Définition**

In [0]:
def testMillerBase(n, a):
  assert n>3, "n doit être supérieur à 3"
  assert a<n, "a doit être inférieur à n"
  
  s, d = resolveMillerFormula(n)

  x = expoRapide(a, d, n)

  if x==1 or x==n-1:
    return False

  for _ in range(s-1):
    x = expoRapide(x, x, n)
    if x==n-1:
      return False
  
  return True

####**Test**

In [0]:
n = 23
a = 6
r = testMillerBase(n, a)
print(n,"est premier :",not r)
print("Comparaison avec la bibliothèque Sympy :", sy.isprime(n))
print("")

n = 561
a = 50
r = testMillerBase(n, a)
print(n,"est premier :",not r)
print("Comparaison avec la bibliothèque Sympy :", sy.isprime(n))
print("Le témoin 50 est un menteur pour l'entier 561.")

23 est premier : True
Comparaison avec la bibliothèque Sympy : True

561 est premier : True
Comparaison avec la bibliothèque Sympy : False
Le témoin 50 est un menteur pour l'entier 561.


##**Question 2**

###**Test de Miller**

####**Définition**

In [0]:
def testMiller(n, m=20):
  if n<=1:
    return False
  if n<=3:
    return True
  if not n%2:
    return False
  
  for _ in range(m):
    a = rand.randint(2,n-2)
    if testMillerBase(n, a):
      return False
  return True

####**Test**

In [0]:
n = 23
r = testMiller(n)
print(n,"est premier :",r)
print("Comparaison avec la bibliothèque Sympy :", sy.isprime(n))
print("")

n = 221
r = testMiller(n)
print(n,"est premier :",r)
print("Comparaison avec la bibliothèque Sympy :", sy.isprime(n))
print("L'utilisation de multiples témoins, permet de mettre en défaut les témoins menteurs'")

23 est premier : True
Comparaison avec la bibliothèque Sympy : True

221 est premier : False
Comparaison avec la bibliothèque Sympy : False
L'utilisation de multiples témoins, permet de mettre en défaut les témoins menteurs'


##**Question 3**

###**Générateur de nombre premier**

####**Définition**

In [0]:
def generateurPremier(k):
  assert k>=2, "k doit être supérieur strictement à 1"
  
  x = 1
  while (not testMiller(x, 100)):
    x = rand.randint(2**(k-1), 2**(k)-1) | 1

  return x

####**Test**

In [0]:
power = [5, 10, 100, 250, 500]#, 1000, 2000, 3000, 4000, 5000]

for k in power:
  t = micro()
  x = generateurPremier(k)
  print(k, "-", x, "est bien premier :",sy.isprime(x))
  print("#Nombre premier trouvé en",displayPeriod(micro()-t))
  print("")

5 - 19 est bien premier : True
#Nombre premier trouvé en 1.308ms

10 - 739 est bien premier : True
#Nombre premier trouvé en 930µs

100 - 959800255371956203200163544867 est bien premier : True
#Nombre premier trouvé en 33.029ms

250 - 1770172546499726130836971998089055782153916255665168288039650346498318302967 est bien premier : True
#Nombre premier trouvé en 136.512ms

500 - 2768788258833570172228392481098896800136903159833011806475625888293615254312864089731066639905916786672558343060647227291663206892125420658966758062707 est bien premier : True
#Nombre premier trouvé en 764.172ms



#**Partie 5 - ElGamal**

##**Construction de p**

###**Question 1**


####**Condition de complexité**

L'algoritme ElGamal repose sur la complexité du problème de logarithme discret (déchiffrement par force brut), contrairement à l'exponentiation.

Le problème de logarithme discret requiert une clé $p$, qui doit être très grande et nombre premier, et $p-1$ avec un très grand facteur premier, d'où $p = q\times2-1$.

###**Question 2**

####**Génération d'une nombre premier ElGamal**

#####**Définition**

In [0]:
def premierElGamal(k):

  q = 0
  p = q*2+1
  while not testMiller(p):
    q = generateurPremier(k)
    p = q*2+1

  return p, q

#####**Test**

In [0]:
power = [2, 5, 10, 100]

for k in power[1:]:
  t = micro()
  p, q = premierElGamal(k)
  print(k, "donne", p,"=", q,"* 2 + 1")
  print(p,"est premier :",sy.isprime(x))
  print("#Nombre premier de ElGamal trouvé en",displayPeriod(micro()-t))
  print("")

5 donne 47 = 23 * 2 + 1
47 est premier : True
#Nombre premier de ElGamal trouvé en 7.397ms

10 donne 1367 = 683 * 2 + 1
1367 est premier : True
#Nombre premier de ElGamal trouvé en 8.668ms

100 donne 2304558783229609848584708612159 = 1152279391614804924292354306079 * 2 + 1
2304558783229609848584708612159 est premier : True
#Nombre premier de ElGamal trouvé en 224.775ms



##**Construction d'une clé (p,a)**

###**Question 1**

####**Clé Privé ElGamal**

#####**Définition**

In [0]:
def clePriveElGamal(k):

  p, _ = premierElGamal(k)
  a = rand.randint(0, p-2)

  return p, a

#####**Test**

In [0]:
k = 100
p, a = clePriveElGamal(k)
print("Génération d'une clé privé ElGamal (p,a) :", (p, a))

Génération d'une clé privé ElGamal (p,a) : (1650959122451780791159709751599, 354739514221855586739284007364)


####**Clé Public ElGamal**

#####**Définition**

In [0]:
def clePublicElGamal(p, a):
  
  m = rand.randint(0, p-1)
  n = expoRapide(m, a, p)

  return p, m, n

#####**Test**

In [0]:
_, m, n = clePublicElGamal(p, a)
print("Génération d'une clé public ElGamal (p,m,n) :", (p, m, n))

Génération d'une clé public ElGamal (p,m,n) : (1650959122451780791159709751599, 893399658037006699894253782720, 1039934498349289515595616355709)


##**Implémentation de ElGamal**

###**Chiffrement**

####**Définition**

In [0]:
def chiffrementElGamal(x, public):

  p, m, n = public
  assert x<p, "x doit être inférieur à p"

  k = rand.randint(0, p-1)

  y1 = expoRapide(m, k, p)
  y2 = x*expoRapide(n, k, p)

  return y1, y2

####**Test**

In [0]:
msg = "Hello World !"
print("Le message est : \""+msg+"\"")
print("")

print("Un bloc est de taille maximum :",tailleBlocMax(p))
print("")

cMsg = encodageBloc(msg)

elGamal = chiffrementElGamal(cMsg, (p, m, n))
print("Le bloc chiffré donne :", elGamal)

Le message est : "Hello World !"

Un bloc est de taille maximum : 15

Le bloc chiffré donne : (1306388977984388828495809868286, 2025254192777747783834750515641239306814957186967630612)


###**Déchiffrement**

####**Définition**

In [0]:
def dechiffrementElGamal(y, prive):

  y1, y2 = y
  p, a = prive
  x = (expoRapide(y1, p-1-a, p) * y2) %p

  return x

####**Test**

In [0]:
print("Le message à déchiffré est :", elGamal)
print("")

dMsg = dechiffrementElGamal(elGamal, (p, a))
print("Le bloc déchiffré donne :", dMsg)
print("")

newMsg = decodageBloc(dMsg)
print("Le message déchiffré et décodé donne :",newMsg)
print("                     Message initial :",msg)

cMsg = encodageBloc(msg)

Le message à déchiffré est : (1306388977984388828495809868286, 2025254192777747783834750515641239306814957186967630612)

Le bloc déchiffré donne : 6379322887179225045360026

Le message déchiffré et décodé donne : Hello World !
                     Message initial : Hello World !


###**Chiffrement ElGamal d'un message entier**

####**Définition**

In [0]:
def messageToElGamal(msg, public):

  p, _, _ = public

  taille = tailleBlocMax(p)
  l = encodage(msg, tailleBloc=taille)

  result = []
  for elt in l:
    result.append(chiffrementElGamal(elt, public))
  
  return result

####**Test**

In [0]:
msg = "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
msgElGamal = messageToElGamal(msg, (p, m, n))
print("Le message \""+msg+"\"")
print("à été chiffré :",msgElGamal)

Le message "à Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque egestas eget orci sit amet ullamcorper. Curabitur venenatis non nulla eu ullamcorper. Sed dignissim, libero sed sodales posuere, sapien enim eleifend nisi, sed fringilla turpis tortor sit amet nibh. Nam at arcu id massa aliquam tincidunt. Aliquam vulputate, sapien id pellentesque sodales, leo magna ullamcorper urna, non rutrum erat lorem nec justo. In varius mauris neque, at ultrices ipsum bibendum at. Aliquam vel molestie dolor, sed mattis nisl. Maecenas auctor quis tortor a placerat. Suspendisse lobortis pharetra nunc, eu iaculis justo tristique at. Morbi augue purus, elementum non posuere ac, pulvinar sit amet odio. Nulla vitae. "
à été chiffré : [(579404446854333292151948965988, 5824333498527488880456611885547362083871356444620867018698), (123303295521095914271839918720, 1352438666949683796103388637377631403765417667247701959047), (166375184691107124225939259433, 32987516041316435438662370921125717410876

###**Déchiffrement ElGamal d'un message**

####**Définition**

In [0]:
def elGamalToMessage(elGamal, prive):

  result = []
  for elt in elGamal:
    result.append(dechiffrementElGamal(elt, prive))
  
  p, _ = prive
  taille = tailleBlocMax(p)
  l = decodage(result, tailleBloc=taille)

  return l

####**Test**

Le message est donc bien déchiffré, exepté, encore une fois, pour le caractère spécial "à" qui a été remplacé par un "@".

In [0]:
print("Le message à déchiffrer est :",msgElGamal)
print("")

msgAfter = elGamalToMessage(msgElGamal, (p, a))
print("Message déchiffré :",msgAfter)
print("     Vérification :",msg)

Le message à déchiffrer est : [(579404446854333292151948965988, 5824333498527488880456611885547362083871356444620867018698), (123303295521095914271839918720, 1352438666949683796103388637377631403765417667247701959047), (166375184691107124225939259433, 3298751604131643543866237092112571741087602355753914740935), (1167945312340171595554151539906, 1152030657539601863664117145600461031374786814729665850796), (439091179185528052793746112132, 90189170894798702713395081345011679634216660958688763977), (1600906996560228350888825428175, 2237676580545123219330790030816731573476040946443037963950), (938797318257864117798761889018, 6641462460162621238585574203730008083742128450048121628672), (289307106257941572172154531529, 10765769290140604218704671454961638206523561634111267121900), (1439972791382544295399202845977, 4932181436241281812988367031203003451510847472474753823907), (34219396494374690376605451694, 3835914281799206531354200900739811493622641557576666450877), (829651522732698075188763080

#**Partie 6 - Arithmétique dans K[X] et application à AES**

##**Question 1**

###**Implémentation de K[X] dans un corps**

Implémentation de la fonction $inverseMod(x, n)$ pour $K[X]$ est demmandé Question 2 mais est requis pour l'implémentation de la divison de $PolynomeModRing$

####**Définition**

Cette implémentation concerne les polynômes à une variable, par exemple : $2x^3+4x^2+5x+10$.

Les polynomes sont à coéfficients dans $F_n[X]$. Ces coefficients sond donc des entiers $a_k$ tel que 0 $\leq$ $a_k$ $\lt$ n.

In [0]:
class PolynomeModRing:

  #Constructor
  def __init__(self, p, n):
    assert isinstance(p, list), "p doit être sous forme de liste de coefficients"
    assert isinstance(n, int), "n doit être un entier"

    self.p = [p[i]%n for i in range(len(p))]
    self.n = n
  
  def __neg__(self):
    return PolynomeModRing([-k for k in self.p], self.n)

  def __sub__(self, pq):
    assert isinstance(pq, PolynomeModRing), "k doit être un polynome"
    #assert self.n==pq.n, "le modulus des deux polynômes doivent être identiques"

    return self+(-pq)

  # Redéfinition de l'addition
  def __add__(self, pq): 
    assert isinstance(pq, PolynomeModRing), "k doit être un polynome"
    assert self.n==pq.n, "le modulus des deux polynômes doivent être identiques"

    p = copy.copy(self)
    q = copy.copy(pq)
    if (p.size()<q.size()):
      p.set([0]*(q.size()-p.size())+p.p)
    elif (p.size()>q.size()):
      q.set([0]*(p.size()-q.size())+q.p)

    r = [(p.p[i]+q.p[i])%self.n for i in range(p.size())]

    return PolynomeModRing(r, self.n)

  # Redéfinition de la multiplication
  def __mul__(self, pq):
    assert isinstance(pq,int) or isinstance(pq, PolynomeModRing), "q doit être un entier ou un polynôme"
  
    if isinstance(pq, PolynomeModRing):
      if pq.powerMax() == 0:
        pq = pq.p[pq.size()-1]

    if isinstance(pq, int):
      r = [(self.p[i]*pq)%self.n for i in range(self.size())]
      return PolynomeModRing(r, self.n)
    
    if isinstance(pq, PolynomeModRing):
      assert self.n==pq.n, "le modulus des deux polynômes doivent être identiques"
      
      p = copy.copy(self)
      q = copy.copy(pq)
      if (p.size()<q.size()):
        p.set([0]+p.p)
      else:
        q.set([0]+q.p)
      
      r = []
      for i in range(q.size()):
        r0 = (p*q.p[i])
        r0.set(r0.p + [0]*(q.size()-1-i))
        r.append(r0)
      
      tailles = [rk.size() for rk in r]
      r2 = PolynomeModRing([0]*max(tailles),self.n)
      for ri in r:
        r2 += ri

      r2.simplify()

      return r2

  # Redéfinition de la multiplication
  def __floordiv__(self, pq):
    assert isinstance(pq,int) or isinstance(pq, PolynomeModRing), "q doit être un entier ou un polynôme"
  
    if isinstance(pq, PolynomeModRing):
      if pq.powerMax() == 0:
        pq = pq.p[pq.size()-1]
    
    if isinstance(pq, int):
      r = [(self.p[i]*inverseMod(pq, self.n))%self.n for i in range(self.size())]
      print(r)
      return PolynomeModRing(r, self.n).simplify()

    if isinstance(pq, PolynomeModRing):
      assert self.n==pq.n, "le modulus des deux polynômes doivent être identiques"

      p = copy.copy(self)
      q = copy.copy(pq)
      if (p.size()<q.size()):
        p.set([0]*(q.size()-p.size())+p.p)
      elif (p.size()>q.size()):
        q.set([0]*(p.size()-q.size())+q.p)
      
      first = True
      r = PolynomeModRing([0], self.n)
      while p.powerMax()>=q.powerMax():
        rkPow = p.powerMax()-q.powerMax()
        rkCoef = p.p[p.size()-p.powerMax()-1]*inverseMod(q.p[q.size()-q.powerMax()-1], self.n)%self.n
        rk = PolynomeModRing([0]*(rkPow+1), self.n)
        rk.p[rk.size()-rkPow-1] = rkCoef
        if first:
          first = False
          r = PolynomeModRing([0]*(rkPow+1), self.n)
        r.p[r.size()-rkPow-1] = rkCoef
        p = p-(q*rk)
      
      return r

  # Redéfinition de la puissance
  def __pow__(self, k):
    return expoRapide(self, k, self.n)

  def powerMax(self):
    i = 0
    while self.size() > 1 and i<self.size()-1 and self.p[i]==0:
      i+=1
    return self.size()-1-i

  def simplify(self):
    while self.size() > 1 and self.p[0]==0:
      self.set(self.p[1:])
    return self
  
  def set(self, p):
    self.p = p

  def size(self):
    return len(self.p)

  def __str__(self):
    return self.toString()

  def toString(self):
    #return str(self.p[len(p)-1])

    if self.powerMax() == 0:
      return "("+str(self.p[self.size()-1])+")"

    m = []
    for k in range(self.size()):
      if (self.p[k] != 0):
        m.append((self.p[k],self.size()-1-k))
    
    return "("+" + ".join([str(m[i][0])+"x^"+str(m[i][1]) for i in range(len(m))])+")"

print("Done")

Done


####**Test**

In [0]:
n = 29
p = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15]
q = [-1, 0, 15]

P = PolynomeModRing(p, n)
Q = PolynomeModRing(q, n)

R = P+Q
print("Addition de deux polynômes P+Q :", P,"+", Q,"\n=", R)
print("")

k = 2
R = P*k
print("Produit d'un polynôme et d'un entier P*2 :",P,"*",k,"\n=",R)
print("")

R = P*Q
print("Produit de deux polynômes P*Q :", P, "*", Q, "\n=",R)
print("")

R = P//Q
print("Division euclidienne de deux polynômes P//Q=R :", P, "//", Q, "\n=",R)

Addition de deux polynômes P+Q : (1x^10 + 15x^0) + (28x^2 + 15x^0) 
= (1x^10 + 28x^2 + 1x^0)

Produit d'un polynôme et d'un entier P*2 : (1x^10 + 15x^0) * 2 
= (2x^10 + 1x^0)

Produit de deux polynômes P*Q : (1x^10 + 15x^0) * (28x^2 + 15x^0) 
= (28x^12 + 15x^10 + 14x^2 + 22x^0)

Division euclidienne de deux polynômes P//Q=R : (1x^10 + 15x^0) // (28x^2 + 15x^0) 
= (28x^8 + 14x^6 + 7x^4 + 18x^2 + 9x^0)


##**Question 2**

###**Euclide Etendu dans $F_n[X]$**

####**Re-Définition**

In [0]:
def euclideEtendu(a, b):
  
  r, r0 = a, b
  u, v, u0, v0 = None, None, None, None

  if isinstance(a, int):
    u, v, u0, v0 = 1, 0, 0, 1
  elif isinstance(a, PolynomeModRing):
    u, v, u0, v0 = PolynomeModRing([1], a.n), PolynomeModRing([0], a.n), PolynomeModRing([0], a.n), PolynomeModRing([1], a.n)
  else:
    raise Exception("TypeError","Cette fonction a été définie pour des paramètres de type entiers ou polynomes")

  while (isinstance(r0, int) and r0) or (isinstance(r0, PolynomeModRing) and (r0.powerMax()!=0 or r0.p[r0.size()-1])):
    q = r//r0
    r, u, v, r0, u0, v0 = r0, u0, v0, r-(q*r0), u-(q*u0), v-(q*v0)

  return r, u, v

####**Test**

In [0]:
R, U, V = euclideEtendu(P, Q)

print("Le PGCD de", P, "et de", Q, "est :",R)
print("")
print("De plus : R=P*U+Q*V avec U =",U," et V =",V)
print("")
print("Verification :",P,"*",U,"+",Q,"*",V,"\n=",P*U+Q*V)



[22, 0, 18]
Le PGCD de (1x^10 + 15x^0) et de (28x^2 + 15x^0) est : (25)

De plus : R=P*U+Q*V avec U = (1)  et V = (1x^8 + 15x^6 + 22x^4 + 11x^2 + 20x^0)

Verification : (1x^10 + 15x^0) * (1) + (28x^2 + 15x^0) * (1x^8 + 15x^6 + 22x^4 + 11x^2 + 20x^0) 
= (25)


####**Inverse Modulaire dans $F_n[X]$**

####**Définition**

In [0]:
def inverseMod(x, n):
  r, u, _ = euclideEtendu(x, n)
  
  if isinstance(r, int):
    assert r==1, "x et n doivent être premiers entre eux"

  if isinstance(r, PolynomeModRing):
    assert r.powerMax() and r.p[r.size()-1]==[1], "x et n doivent être premiers entre eux"

  return u%n

####**Test**

In [0]:
#NE FONCTIONNE PAS

#N = PolynomeModRing([n], n+1)

#R = inverseMod(P, N)
#print("L'inverse de",P,"modulo",n,"est :",R)

##**Question 2**

###**Utilisation de cette implémentation dans la routine SubBytes d'AES**

Cette partie du chiffrement AES consiste à effectuer des clalculs sur des bits sous forme de polynôme, il faudra donc utiliser pour chaque octet $PolynomeModRing([a_7,a_6,a_5,a_4,a_3,a_2,a_1, a_0], 2)$. Ensuite il sera donc possible d'effectuer les opérations sur ces polynômes afin d'obtenir la matrice de transformation afin de l'application sur chaque octet de l'entrée.